In [1]:
import cv2
import os
import numpy as np
import tensorflow as tf


print(tf.test.is_gpu_available())
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

C:\Users\witlab\anaconda3\envs\gpu_test_python3.6\lib\site-packages\tensorflow\python\framework\dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\witlab\anaconda3\envs\gpu_test_python3.6\lib\site-packages\tensorflow\python\framework\dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\witlab\anaconda3\envs\gpu_test_python3.6\lib\site-packages\tensorflow\python\framework\dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\witlab\anaconda3\envs\

True
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1741243993258192414
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 78607156
locality {
  bus_id: 1
}
incarnation: 14952660512657211156
physical_device_desc: "device: 0, name: GeForce GTX 1050 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [2]:
# image_path = r'image\auto_intg\preprocess\train_image_20240228'
image_path=r'image\auto_intg\preprocess_no_cut\train_image_20240306'
image_files = [f for f in os.listdir(image_path) if f.endswith('.jpg')]

images = []
for file in image_files:
    img = cv2.imread(os.path.join(image_path, file))
    if img is None:
        print(f"{file} 읽을 수 없음")
        continue

    img = img / 255.0  # 0~1로 정규화
    images.append(img)

images = np.array(images)
print(images.shape)

(6615, 224, 224, 3)


In [3]:
import pandas as pd

excel_path = 'data/train_data_20240214-20240227_20240228.xlsx'
df = pd.read_excel(excel_path, engine='openpyxl')

illum_columns = [f'illum_{i}' for i in range(1, 10)]

# Extract output (illum) columns
target_data = (df[illum_columns]).to_numpy()
target_data[target_data < 0] = 0
print(target_data.shape)

(6615, 9)


In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(images, target_data, test_size=0.2, random_state=42)

print("Training data shape:", X_train.shape)
print("Testing data shape:", X_test.shape)
print("Training target shape:", y_train.shape)
print("Testing target shape:", y_test.shape)

Training data shape: (5292, 224, 224, 3)
Testing data shape: (1323, 224, 224, 3)
Training target shape: (5292, 9)
Testing target shape: (1323, 9)


In [5]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, AveragePooling2D, UpSampling2D
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

model = Sequential()
model.add(Conv2D(64, (3,3), activation='relu', padding='same', input_shape=(224, 224, 3)))
model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(MaxPooling2D())

model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(MaxPooling2D())

model.add(Conv2D(256, (3,3), activation='relu', padding='same'))
model.add(Conv2D(256, (3,3), activation='relu', padding='same'))
model.add(MaxPooling2D())

model.add(Conv2D(512, (3,3), activation='relu', padding='same'))
model.add(Conv2D(512, (3,3), activation='relu', padding='same'))
model.add(MaxPooling2D())

# Bottleneck
model.add(Conv2D(1024, (3,3), activation='relu', padding='same'))
model.add(Conv2D(1024, (3,3), activation='relu', padding='same'))

# Decoder
model.add(UpSampling2D())
model.add(Conv2D(512, (3,3), activation='relu', padding='same'))
model.add(Conv2D(512, (3,3), activation='relu', padding='same'))

model.add(UpSampling2D())
model.add(Conv2D(256, (3,3), activation='relu', padding='same'))
model.add(Conv2D(256, (3,3), activation='relu', padding='same'))

model.add(UpSampling2D())
model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(Conv2D(128, (3,3), activation='relu', padding='same'))

model.add(UpSampling2D())
model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(Flatten())
model.add(Dense(9))

# Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['accuracy'])
model.summary()


early_stopping_cb = EarlyStopping(patience =10, monitor = 'val_loss')
checkpoint_cb = ModelCheckpoint("model/unet_best_model_img_no_cut_20240306.h5", monitor='val_loss', save_best_only=True, mode='min')


history = model.fit(X_train, y_train, epochs=500, batch_size=2, validation_data=(X_test, y_test), callbacks=[early_stopping_cb,checkpoint_cb])

# Evaluate the model on the test set
test_loss = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 56, 56, 256)       295168    
__________

ResourceExhaustedError: OOM when allocating tensor with shape[32,128,224,224]
	 [[Node: conv2d_17/Conv2D = Conv2D[T=DT_FLOAT, data_format="NHWC", padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](up_sampling2d_4/ResizeNearestNeighbor, conv2d_16/kernel/read)]]
	 [[Node: loss/mul/_297 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_264_loss/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'conv2d_17/Conv2D', defined at:
  File "C:\Users\witlab\anaconda3\envs\gpu_test_python3.6\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\witlab\anaconda3\envs\gpu_test_python3.6\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\witlab\anaconda3\envs\gpu_test_pytho

In [ ]:
os.environ['KMP_DUPLICATE_LIB_OK']='True'

import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train', 'val'])
plt.show()

### 2024.02.21 데이터로 성능 테스트

In [ ]:
test_image_path = r'image\auto_intg\preprocess_no_cut\20240221'
test_image_files = [f for f in os.listdir(test_image_path) if f.endswith('.jpg')]

test_images = []
for file in test_image_files:
    img = cv2.imread(os.path.join(test_image_path, file))
    if img is None:
        print(f"{file} 읽을 수 없음")
        continue

    img = img / 255.0  # 0~1로 정규화
    test_images.append(img)

test_images = np.array(test_images)
print(test_images.shape)

In [ ]:
# 2024.02.21 조도 데이터 불러오기

test_excel_path = 'data/test_data_20240221.xlsx'
test_df = pd.read_excel(test_excel_path, engine='openpyxl')

illum_columns = [f'illum_{i}' for i in range(1, 10)]

# Extract output (illum) columns
test_target_data = (test_df[illum_columns]).to_numpy()
test_target_data[test_target_data < 0] = 0
print(test_target_data.shape)

In [ ]:
pred = model.predict(test_images)

In [ ]:
print(test_target_data.shape)
print(pred.shape)
print(type(test_target_data))
print(type(pred))

In [ ]:
# Create DataFrames from the arrays
actual_df = pd.DataFrame(test_target_data, columns=[f'Actual_Illum_{i+1}' for i in range(9)])
predicted_df = pd.DataFrame(pred, columns=[f'Predicted_Illum_{i+1}' for i in range(9)])

# Concatenate the DataFrames horizontally
comparison_data = pd.concat([actual_df, predicted_df], axis=1)

# Save the DataFrame to an Excel file
excel_output_path = 'unet_results_20240221_test_20240318_img_no_cut.xlsx'
comparison_data.to_excel(excel_output_path, index=False)

print(f"Comparison results saved to {excel_output_path}")

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error
r_squared = r2_score(test_target_data, pred)
mae = mean_absolute_error(test_target_data, pred)

print("Mean Absolute Error (MAE):", mae)
print("R-squared (R²):", r_squared)